In [100]:
'''Solve the Sequence Alignment with Profile HMM Problem'''
### approach #1 use shape of Trans matrix 


class HMM_pseudo:
    import pandas as pd
    import numpy as np

    def __init__(self, threshold, pseudo, alphabet, alignment):
        self.th = threshold #e.g. 0.301
        self.pseudo = pseudo # e.g. 0.01
        self.alph = alphabet #e.g. ['A', 'B', 'C', 'D', 'E']
        self.aln = alignment #e.g.  [['E', 'B', 'A'], ['E', '-', 'D'], ['E', 'B', '-'], ['E', 'E', 'D'], ['E', 'B', 'D'], ['E', 'B', 'E'], ['E', '-', 'D'], ['E', '-', 'D']]
        self.I = self.find_insertions()[1]
        self.good_col = self.find_insertions()[0]
        self.hmm = ['S','I0']+[St+str(n) for n in range(1,self.good_col+1) for St in ['M','D','I']]+['E']
    
    def find_insertions(self):
        a=pd.DataFrame(self.aln)
        I={}
        good_col=len(a.columns)
        M_counter=0
        for n in range(len(a.columns)):
            col=n
            try:
                if float(a[col].value_counts().to_dict()['-'])/sum(a[col].value_counts())>=th:
                    try:
                        assert I[n-1]
                        I[n]=I[n-1]
                    except KeyError:
                        I[n]='I'+str(M_counter)
                    good_col-=1
                else:
                    M_counter+=1
            except KeyError:
                M_counter+=1
        return good_col,I
    
    def transition_matrix(self):
        Trans=pd.DataFrame(0,index=self.hmm,columns=self.hmm)
        pseudo=self.pseudo
        I=self.I
        aln=self.aln
        good_col=self.good_col
        for n in range(len(aln)):
            seq=aln[n]
            M_counter=1
            path=['S']
            for i in range(len(seq)):
                if i in I.keys():
                    if seq[i]=='-':
                        continue
                    else:
                        if path[-1][0]=='I':
                            path+=[path[-1]]
                            Trans[path[-1]][path[-2]]+=1
                            continue
                        else:
                            path+=['I'+str(M_counter-1)]
                            Trans[path[-1]][path[-2]]+=1
                elif seq[i] =='-':
                    path+=['D'+str(M_counter)]
                    Trans[path[-1]][path[-2]]+=1
                    M_counter+=1
                else:
                    path+=['M'+str(M_counter)]
                    Trans[path[-1]][path[-2]]+=1
                    M_counter+=1
            path+=['E']
            Trans[path[-1]][path[-2]]+=1
        
        ### ger rel, add pseudocounts and get rel ###
        ## start
        subset=Trans.loc['S':'I0', 'I0':'D1']
        for row in subset.index:
            subset.loc[row]=subset.loc[row]/subset.loc[row].sum()
            subset.loc[row]=subset.loc[row].fillna(0)
            subset.loc[row]=(subset.loc[row]+pseudo)/(subset.loc[row].sum()+3*pseudo)
        Trans.loc['S':'I0', 'I0':'D1']=subset
        ## between matches
        for n in range(1,good_col):
            subset=Trans.loc['M'+str(n):'I'+str(n), 'I'+str(n):'D'+str(n+1)]
            for row in subset.index:
                subset.loc[row]=subset.loc[row]/subset.loc[row].sum()
                #print 'subset.loc[row]', row
                #print subset.loc[row]
                subset.loc[row]=subset.loc[row].fillna(0)
                subset.loc[row]=(subset.loc[row]+pseudo)/(subset.loc[row].sum()+3*pseudo)
            Trans.loc['M'+str(n):'I'+str(n), 'I'+str(n):'D'+str(n+1)]=subset
        ## end
        subset=Trans.loc['M'+str(good_col):'I'+str(good_col), 'I'+str(good_col):'E']
        for row in subset.index:
            subset.loc[row]=subset.loc[row]/subset.loc[row].sum()
            subset.loc[row]=subset.loc[row].fillna(0)
            subset.loc[row]=(subset.loc[row]+pseudo)/(subset.loc[row].sum()+2*pseudo)
        Trans.loc['M'+str(good_col):'I'+str(good_col), 'I'+str(good_col):'E']=subset
        return Trans
    
    def emission_matrix(self):
        Em=pd.DataFrame(0,index=self.hmm,columns=self.alph)
        I=self.I
        aln=self.aln
        for n in range(len(aln)):
            seq=aln[n]
            M_counter=1
            for i in range(len(seq)):
                if i in I.keys():
                    if seq[i]=='-':
                        continue
                    else:
                        Em[seq[i]][I[i]]+=1
                elif seq[i]=='-':
                    M_counter+=1
                    pass
                else:
                    Em[seq[i]]['M'+str(M_counter)]+=1
                    M_counter+=1
        for row in Em.index:
            if row[0] in ['S','D','E']:
                pass
            else:
                Em.loc[row]=Em.loc[row]/Em.loc[row].sum()
                Em.loc[row]=Em.loc[row].fillna(0)
                Em.loc[row]=(Em.loc[row]+pseudo)/(Em.loc[row].sum()+len(self.alph)*pseudo)
        return Em  

import pandas as pd
import numpy as np

file='input.txt'
#file='input2.txt'
#file='../../Downloads/dataset_11632_4.txt'
#file='../../Downloads/rosalind_ba10f.txt'

f=open(file,'r')
lines=f.read().splitlines()
f.close()

x=list(lines[0])
th=float(map(str,lines[2].split(' '))[0])
pseudo=float(map(str,lines[2].split(' '))[-1])
alph=map(str,lines[4].split(' '))
aln=[list(line) for line in lines[6:]]

print th,pseudo,alph,aln
Trans= HMM_pseudo(th,pseudo,alph,aln).transition_matrix()
Trans=np.log10(Trans)
#HMM_pseudo(th,pseudo,alph,aln).transition_matrix().to_csv('output.txt', mode='w', header=True,sep='\t')

Em=HMM_pseudo(th,pseudo,alph,aln).emission_matrix()
Em=np.log10(Em)
Em.loc['S']=[0]*len(alph)

#HMM_pseudo(th,pseudo,alph,aln).emission_matrix().to_csv('output2.txt', mode='w', header=True,sep='\t')

gc=HMM_pseudo(th,pseudo,alph,aln).good_col
hmm=HMM_pseudo(th,pseudo,alph,aln).hmm
print
print x
print
print Em
#print st
st=list(Trans.columns)
#print st
Pi={}
Backtrack={}
d={}
pointer=1
posi=0
for n in range(pointer,pointer+3):
    #print State
    State=st[n]
    Pi[State]={'sc':float,'pred':'','posi':0}
    predecessors=[st[i] for i in range(pointer-1,pointer+1)]
    if State[0]=='D':
        score=max([Trans[State][Pred] for Pred in predecessors])
        pred=[Pred for Pred in predecessors if Trans[State][Pred]==score][0]
        Pi[State].update({'sc':score,'pred':pred,'posi':0})

    else:
        score=max([Trans[State][Pred]+Em[x[posi]][State] for Pred in predecessors])
        pred=[Pred for Pred in predecessors if Trans[State][Pred]+Em[x[0]][State]==score][0]
        Pi[State].update({'sc':score,'pred':pred,'posi':0})
print Pi        
pointer+=3
while st[pointer+1] != 'E':
    for n in range(pointer,pointer+3):
        State=st[n]
        if State not in Pi.keys():
            Pi[State]={'sc':-float('Inf'),'pred':'','posi':0}
        predecessors=[st[i] for i in range(pointer-2,pointer+1)]
       
        #print
        print State,predecessors
        if State[0]=='D':
            for Pred in predecessors:
                try:
                    x[Pi[Pred]['posi']+1]
                except IndexError:
                    predecessors.remove(Pred)
            score=max([Pi[Pred]['sc']+Trans[State][Pred] for Pred in predecessors])
            pred=[Pred for Pred in predecessors if Pi[Pred]['sc']+Trans[State][Pred]==score][0]
            Pi[State]={'sc':score,'pred':pred,'posi':Pi[pred]['posi']}
        else:
            for Pred in predecessors:
                try:
                    x[Pi[Pred]['posi']+1]
                    if len(x)-(Pi[Pred]['posi']+1) > len(aln[0])-int(State[1:]):
                        pass
                        #print 'Pred',Pred,"nuc to fit",len(x)-Pi[Pred]['posi']-1,'space avail',len(aln[0])-int(State[1:])
                        #predecessors.remove(Pred)
                except IndexError:
                    predecessors.remove(Pred)
            #print predecessors
            #print [x[Pi[Pred]['posi']+1] for Pred in predecessors]
            if State=='M6':
                print [[Pi[Pred]['sc'],Trans[State][Pred],Em[x[Pi[Pred]['posi']+1]][State]] for Pred in predecessors]
                print len(aln[0]),int(State[1:])
                #print [[len(x),Pi[Pred]['posi'],gc,int(State[1:])] for Pred in predecessors]
            score=max([Pi[Pred]['sc']+Trans[State][Pred]+Em[x[Pi[Pred]['posi']+1]][State] for Pred in predecessors])
            pred=[Pred for Pred in predecessors if Pi[Pred]['sc']+Trans[State][Pred]+Em[x[Pi[Pred]['posi']+1]][State]==score][0]
            Pi[State]={'sc':score,'pred':pred,'posi':Pi[pred]['posi']+1}
        print Pi[State]

    pointer+=3

#print Pi
#print pointer

for n in range(pointer,pointer+2):
    State=st[n]
    predecessors=[st[i] for i in range(pointer-2,pointer+1)]
    if State not in Pi.keys():
            Pi[State]={'sc':-float('Inf'),'pred':'','posi':0}
    #Pi[State]={'sc':float,'pred':'','posi':int}   
    print
    print State,predecessors
    
    for Pred in predecessors:
        try:
            print Pred,Pi[Pred]['posi']
            x[Pi[Pred]['posi']+1]
        except IndexError:
            predecessors.remove(Pred)
    print predecessors
    print [x[Pi[Pred]['posi']+1] for Pred in predecessors]
    score=max([Pi[Pred]['sc']+Trans[State][Pred]+Em[x[Pi[Pred]['posi']+1]][State] for Pred in predecessors])
    pred=[Pred for Pred in predecessors if Pi[Pred]['sc']+Trans[State][Pred]+Em[x[Pi[Pred]['posi']+1]][State]==score][0]
    Pi[State]={'sc':score,'pred':pred,'posi':Pi[pred]['posi']+1}

    
result=[]
node='E'
posi=Pi[node]['posi']
while node != 'S':
    prenode=Pi[node]['pred']
    result+=[prenode]
    posi=Pi[prenode]['posi']
    node=prenode
    print prenode,posi
print
print result[::-1]

0.4 0.01 ['A', 'B', 'C', 'D', 'E', 'F'] [['A', 'C', 'D', 'E', 'F', 'A', 'C', 'A', 'D', 'F'], ['A', 'F', 'D', 'A', '-', '-', '-', 'C', 'C', 'F'], ['A', '-', '-', 'E', 'F', 'D', '-', 'F', 'D', 'C'], ['A', 'C', 'A', 'E', 'F', '-', '-', 'A', '-', 'C'], ['A', 'D', 'D', 'E', 'F', 'A', 'A', 'A', 'D', 'F']]

['A', 'E', 'F', 'D', 'F', 'D', 'C']

           A         B         C         D         E         F
S   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
I0 -0.778151 -0.778151 -0.778151 -0.778151 -0.778151 -0.778151
M1 -0.020984 -2.025306 -2.025306 -2.025306 -2.025306 -2.025306
D1      -inf      -inf      -inf      -inf      -inf      -inf
I1 -0.778151 -0.778151 -0.778151 -0.778151 -0.778151 -0.778151
M2 -2.025306 -2.025306 -0.317736 -0.610333 -2.025306 -0.610333
D2      -inf      -inf      -inf      -inf      -inf      -inf
I2 -0.778151 -0.778151 -0.778151 -0.778151 -0.778151 -0.778151
M3 -0.610333 -2.025306 -2.025306 -0.144492 -2.025306 -2.025306
D3      -inf      -inf      -in

KeyError: 'S'

In [ ]:
### approach #2 dynamic programing with three matrices

import pandas as pd
import numpy as np

file='input.txt'
#file='input2.txt'
#file='../../Downloads/dataset_11632_6.txt'
#file='../../Downloads/rosalind_ba10g.txt'

f=open(file,'r')
lines=f.read().splitlines()
f.close()

x=list(lines[0])
th=float(map(str,lines[2].split(' '))[0])
pseudo=float(map(str,lines[2].split(' '))[-1])
alph=map(str,lines[4].split(' '))
aln=[list(line) for line in lines[6:]]
#print th,pseudo,alph,aln

Trans= HMM_pseudo(th,pseudo,alph,aln).transition_matrix()
Trans=np.log10(Trans)
Em=HMM_pseudo(th,pseudo,alph,aln).emission_matrix()
Em=np.log10(Em)
gc=HMM_pseudo(th,pseudo,alph,aln).good_col
print 'gc',gc
#print HMM_pseudo(th,pseudo,alph,aln).I
print x
#print Trans
#print Em

rows = len(Em)/3-1
M=pd.DataFrame(-float('Inf'),index=xrange(gc+1),columns=xrange(len(x)+1))
I=pd.DataFrame(-float('Inf'),index=xrange(gc+1),columns=xrange(len(x)+1))
D=pd.DataFrame(-float('Inf'),index=xrange(gc+1),columns=xrange(len(x)+1))
M.name='M'
I.name='I'
D.name='D'

I[0][0]=0
M[0][0]=0
D[0][0]=0
#btM=pd.DataFrame(str,index=xrange(gc+1),columns=xrange(len(x)+1))
#btI=pd.DataFrame(str,index=xrange(gc+1),columns=xrange(len(x)+1))
#btD=pd.DataFrame(str,index=xrange(gc+1),columns=xrange(len(x)+1))
#print M
index_list = ['S', 'I0']

for i in xrange(0,gc):
    index_list.extend(['M'+str(i), 'D'+str(i), 'I'+str(i)])
index_list.append('E')
#print index_list
for i in index_list:
    if not i in Trans:
        Trans[i] = pd.Series(-float('Inf'), index=Trans.index)
    for j in index_list:
        if not j in Trans[i]:
            Trans[i][j] = -float('Inf')

#print Trans

for i in range(1,len(x)+1):
    if i == 1:
        I[i][0]=I[i-1][0]+Trans['I0']['S']+Em[x[i-1]]['I0']
    else:
        I[i][0]=I[i-1][0]+Trans['I0']['I0']+Em[x[i-1]]['I0']

for j in range(1,gc+1):
    #print gc
    if j==1:
        D[0][j] = D[0][j-1]+Trans['D'+str(j)]['S']
    else:
        D[0][j] = D[0][j-1]+Trans['D'+str(j)]['D'+str(j-1)]
#print 'I'
#print I    
for i in range(1,len(x)+1):
    for j in range(1,gc+1):
        if i==1 and j==1:
            
            M[i][j]=M[0][0]+Trans['M1']['S']+Em[x[0]]['M1']
            
            D[i][j]=I[1][0]+Trans['D1']['I0']
            
            I[i][j]=max(M[i-1][j]+Trans['I'+str(j)]['M'+str(j)]+Em[x[i-1]]['I'+str(j)],
                        D[i-1][j]+Trans['I'+str(j)]['D'+str(j)]+Em[x[i-1]]['I'+str(j)],
                        I[i-1][j]+Trans['I'+str(j)]['I'+str(j)]+Em[x[i-1]]['I'+str(j)])
        else:
            #print i,j
           
            M[i][j]=max(M[i-1][j-1]+Trans['M'+str(j)]['M'+str(j-1)]+Em[x[i-1]]['M'+str(j)],
                        D[i-1][j-1]+Trans['M'+str(j)]['D'+str(j-1)]+Em[x[i-1]]['M'+str(j)],
                        I[i-1][j-1]+Trans['M'+str(j)]['I'+str(j-1)]+Em[x[i-1]]['M'+str(j)])
            #btM[i][j]=[ST.name for ST in[M,D,I] if M[i][j]== ST[i-1][j-1]+Trans['M'+str(j)][ST.name+str(j-1)]+Em[x[i-1]]['M'+str(j)]]+[j-1]
            
            if [i,j]in [[2,4],[3,5],[5,6]]:
                print 'M'+str(j),M[i][j]
                print ['M'+str(j-1),[M[i-1][j-1],Trans['M'+str(j)]['M'+str(j-1)],Em[x[i-1]]['M'+str(j)]],
                       'D'+str(j-1),[D[i-1][j-1],Trans['M'+str(j)]['D'+str(j-1)],Em[x[i-1]]['M'+str(j)]],
                       'I'+str(j-1),[I[i-1][j-1],Trans['M'+str(j)]['I'+str(j-1)],Em[x[i-1]]['M'+str(j)]]]
            D[i][j]=max(M[i][j-1]+Trans['D'+str(j)]['M'+str(j-1)],
                        D[i][j-1]+Trans['D'+str(j)]['D'+str(j-1)],
                        I[i][j-1]+Trans['D'+str(j)]['I'+str(j-1)])
            #btD[i][j]=[ST.name for ST in[M,D,I] if D[i][j]== ST[i-1][j]+Trans['D'+str(j)][ST.name+str(j-1)]]+[j]
            
            if [i,j] in [[2,5],[4,5]]:
                print 'D'+str(j),D[i][j]
            
            I[i][j]=max(M[i-1][j]+Trans['I'+str(j)]['M'+str(j)]+Em[x[i-1]]['I'+str(j)],
                        D[i-1][j]+Trans['I'+str(j)]['D'+str(j)]+Em[x[i-1]]['I'+str(j)],
                        I[i-1][j]+Trans['I'+str(j)]['I'+str(j)]+Em[x[i-1]]['I'+str(j)])
            #btI[i][j]=[ST.name for ST in[M,D,I] if I[i][j]== ST[i][j-1]+Trans['I'+str(j)][ST.name+str(j)]+Em[x[i-1]]['I'+str(j)]]+[j-1]
            
            #if [i,j]==[3,4]:
            #    print 'I'+str(j),M[i-1][j],Trans['I'+str(j)]['M'+str(j)],Em[x[i-1]]['I'+str(j)]
            
#print 'M11',M[11][11],'M10',M[10][10],'D10',D[9][10]

#print 'I'
#print I
#print 'M'
#print M
#print M[len(x)][gc],I[len(x)][gc],D[len(x)][gc]
#print 'D'
#print D
#print 'bt',btI[4],btD[4]


seq=len(x)
st=gc
path=[]
#print 'M4',M[2][4]
#max_val=max(I[seq][st]+Trans['E']['I'+str(st)],M[seq][st]+Trans['E']['M'+str(st)],D[seq][st]+Trans['E']['D'+str(st)])
#node=[ST.name+str(st) for ST in [I,M,D] if ST[seq][st]+Trans['E'][ST.name+str(st)]==max_val]
#print node
while seq>0 :
    #print 'seq,st',seq,st
    max_val=max(I[seq][st],M[seq][st],D[seq][st])
    node=[ST.name+str(st) for ST in [I,M,D] if ST[seq][st]==max_val]
    path+=node
    #print path
    if node[-1][0]=='M':
        seq-=1
        st-=1
    elif node[-1][0]=='D':
        st-=1
    else: 
        seq-=1
answer=path[::-1]
#print ' '.join(answer)


res = []
seq=len(x)
state=gc
max_value = -float('Inf')
max_state = ('', 0, 0)
next_state = 'E'
while seq>0:
    if I[seq][state] > max_value:
        max_value =  I[seq][state]+Trans[next_state]['I'+str(state)]
        max_state = ('I', state, seq)
    if M[seq][state] > max_value:
        max_value =  M[seq][state]+Trans[next_state]['M'+str(state)]
        max_state = ('M', state, seq)
    if D[seq][state] > max_value:
        max_value =  D[seq][state]+Trans[next_state]['D'+str(state)]
        max_state = ('D', state, seq)

    res.insert(0,max_state[0]+str(max_state[1]))
    next_state = res[0]
    if max_state[0] == 'I':
        seq -= 1
    if max_state[0] == 'D':
        state -= 1
    if max_state[0] == 'M':
        seq -= 1
        state -= 1
print ' '.join(res)


In [170]:
### the one that worked: based on Viterbi graph

class HMM_pseudo:
    import pandas as pd
    import numpy as np

    def __init__(self, threshold, pseudo, alphabet, alignment):
        self.th = threshold #e.g. 0.301
        self.pseudo = pseudo # e.g. 0.01
        self.alph = alphabet #e.g. ['A', 'B', 'C', 'D', 'E']
        self.aln = alignment #e.g.  [['E', 'B', 'A'], ['E', '-', 'D'], ['E', 'B', '-'], ['E', 'E', 'D'], ['E', 'B', 'D'], ['E', 'B', 'E'], ['E', '-', 'D'], ['E', '-', 'D']]
        self.I = self.find_insertions()[1]
        self.good_col = self.find_insertions()[0]
        self.hmm = ['S','I0']+[St+str(n) for n in range(1,self.good_col+1) for St in ['M','D','I']]+['E']
    
    def find_insertions(self):
        a=pd.DataFrame(self.aln)
        I={}
        good_col=len(a.columns)
        M_counter=0
        for n in range(len(a.columns)):
            col=n
            try:
                if float(a[col].value_counts().to_dict()['-'])/sum(a[col].value_counts())>=th:
                    try:
                        assert I[n-1]
                        I[n]=I[n-1]
                    except KeyError:
                        I[n]='I'+str(M_counter)
                    good_col-=1
                else:
                    M_counter+=1
            except KeyError:
                M_counter+=1
        return good_col,I
    
    def transition_matrix(self):
        Trans=pd.DataFrame(0,index=self.hmm,columns=self.hmm)
        pseudo=self.pseudo
        I=self.I
        aln=self.aln
        good_col=self.good_col
        for n in range(len(aln)):
            seq=aln[n]
            M_counter=1
            path=['S']
            for i in range(len(seq)):
                if i in I.keys():
                    if seq[i]=='-':
                        continue
                    else:
                        if path[-1][0]=='I':
                            path+=[path[-1]]
                            Trans[path[-1]][path[-2]]+=1
                            continue
                        else:
                            path+=['I'+str(M_counter-1)]
                            Trans[path[-1]][path[-2]]+=1
                elif seq[i] =='-':
                    path+=['D'+str(M_counter)]
                    Trans[path[-1]][path[-2]]+=1
                    M_counter+=1
                else:
                    path+=['M'+str(M_counter)]
                    Trans[path[-1]][path[-2]]+=1
                    M_counter+=1
            path+=['E']
            Trans[path[-1]][path[-2]]+=1
        
        ### ger rel, add pseudocounts and get rel ###
        ## start
        subset=Trans.loc['S':'I0', 'I0':'D1']
        for row in subset.index:
            subset.loc[row]=subset.loc[row]/subset.loc[row].sum()
            subset.loc[row]=subset.loc[row].fillna(0)
            subset.loc[row]=(subset.loc[row]+pseudo)/(subset.loc[row].sum()+3*pseudo)
        Trans.loc['S':'I0', 'I0':'D1']=subset
        ## between matches
        for n in range(1,good_col):
            subset=Trans.loc['M'+str(n):'I'+str(n), 'I'+str(n):'D'+str(n+1)]
            for row in subset.index:
                subset.loc[row]=subset.loc[row]/subset.loc[row].sum()
                #print 'subset.loc[row]', row
                #print subset.loc[row]
                subset.loc[row]=subset.loc[row].fillna(0)
                subset.loc[row]=(subset.loc[row]+pseudo)/(subset.loc[row].sum()+3*pseudo)
            Trans.loc['M'+str(n):'I'+str(n), 'I'+str(n):'D'+str(n+1)]=subset
        ## end
        subset=Trans.loc['M'+str(good_col):'I'+str(good_col), 'I'+str(good_col):'E']
        for row in subset.index:
            subset.loc[row]=subset.loc[row]/subset.loc[row].sum()
            subset.loc[row]=subset.loc[row].fillna(0)
            subset.loc[row]=(subset.loc[row]+pseudo)/(subset.loc[row].sum()+2*pseudo)
        Trans.loc['M'+str(good_col):'I'+str(good_col), 'I'+str(good_col):'E']=subset
        return Trans
    
    def emission_matrix(self):
        Em=pd.DataFrame(0,index=self.hmm,columns=self.alph)
        I=self.I
        aln=self.aln
        for n in range(len(aln)):
            seq=aln[n]
            M_counter=1
            for i in range(len(seq)):
                if i in I.keys():
                    if seq[i]=='-':
                        continue
                    else:
                        Em[seq[i]][I[i]]+=1
                elif seq[i]=='-':
                    M_counter+=1
                    pass
                else:
                    Em[seq[i]]['M'+str(M_counter)]+=1
                    M_counter+=1
        for row in Em.index:
            if row[0] in ['S','D','E']:
                pass
            else:
                Em.loc[row]=Em.loc[row]/Em.loc[row].sum()
                Em.loc[row]=Em.loc[row].fillna(0)
                Em.loc[row]=(Em.loc[row]+pseudo)/(Em.loc[row].sum()+len(self.alph)*pseudo)
        return Em
    
    #HMM_pseudo(th,pseudo,alph,aln).transition_matrix().to_csv('output.txt', mode='w', header=True,sep='\t')
    def Align_to_HMMprofile(self,x):
        import pandas as pd
        import numpy as np

        #x=self.x
        gc=self.good_col
        Trans=HMM_pseudo(self.th,self.pseudo,self.alph,self.aln).transition_matrix()
        Trans=np.log10(Trans)

        Em=HMM_pseudo(self.th,self.pseudo,self.alph,self.aln).emission_matrix()
        Em=np.log10(Em)
        hmm=self.hmm

        V=pd.DataFrame(-float('Inf'),index=hmm[1:-1],columns=range(len(x)))
        Backtrack=pd.DataFrame(list,index=hmm[1:-1],columns=range(len(x)))
        Index=['S']+['D'+str(n+1) for n in range(gc)]
        sdic={}
        Silent={} #Score for 0th column that contains only silent states
        Silent['S']=0
        for n in range(len(Index)):
            sdic[n]=Index[n]
            if n>0:
                Silent[Index[n]]=Silent[Index[n-1]]+Trans[Index[n]][Index[n-1]]
        ### Fill first column of Viterbi graph
        for node in V.index:
            idx=int(node[1:])
            if node[0]=='D':
                if node=='D1':
                    V[0][node]=V[0]['I0']+Trans[node]['I0']
                    Backtrack[0][node]=[-1,'I0']
                else:
                    V[0][node]=max([V[0][ST+str(idx-1)]+Trans[node][ST+str(idx-1)] for ST in ['M','D','I']])
                    Backtrack[0][node]=[0,[ST+str(idx-1) for ST in ['M','D','I'] if V[0][node]==V[0][ST+str(idx-1)]+Trans[node][ST+str(idx-1)]][0]]
            elif node=='I0':
                V[0][node]=Silent['S']+Trans[node]['S']+Em[x[0]][node]
                Backtrack[0][node]=[-1,'S']
            else:
                V[0][node]=Silent[sdic[idx-1]]+Trans[node][sdic[idx-1]]+Em[x[0]][node]
                Backtrack[0][node]=[-1,sdic[idx-1]]

        ### Fill all following columns of Viterbi graph
        for n in V.columns[1:]:
            for node in V.index:
                idx=int(node[1:])
                if node=='I0':
                    V[n][node]=V[n-1][node]+Trans[node][node]+Em[x[n]][node]
                    Backtrack[n][node]=[int(n-1),node]
                elif node=='M1':
                    V[n][node]=V[n-1]['I0']+Trans[node]['I0']+Em[x[n]][node]
                    Backtrack[n][node]=[n-1,'I0']
                elif node=='D1':
                    V[n][node]=V[n]['I0']+Trans[node]['I0']
                    Backtrack[n][node]=[n,'I0']
                elif node[0]=='I':
                    V[n][node]=max([V[n-1][ST+str(idx)]+Trans[node][ST+str(idx)]+Em[x[n]][node] for ST in ['M','D','I']])
                    Backtrack[n][node]=[n-1,[ST+str(idx) for ST in ['M','D','I'] if V[n][node]==V[n-1][ST+str(idx)]+Trans[node][ST+str(idx)]+Em[x[n]][node]][0]]
                elif node[0]=='M':
                    V[n][node]=max([V[n-1][ST+str(idx-1)]+Trans[node][ST+str(idx-1)]+Em[x[n]][node] for ST in ['M','D','I']])
                    Backtrack[n][node]=[n-1,[ST+str(idx-1) for ST in ['M','D','I'] if V[n][node]==V[n-1][ST+str(idx-1)]+Trans[node][ST+str(idx-1)]+Em[x[n]][node]][0]]
                elif node[0]=='D':
                    V[n][node]=max([V[n][ST+str(idx-1)]+Trans[node][ST+str(idx-1)] for ST in ['M','D','I']])
                    Backtrack[n][node]=[n,[ST+str(idx-1) for ST in ['M','D','I'] if V[n][node]==V[n][ST+str(idx-1)]+Trans[node][ST+str(idx-1)]][0]]

        ### Backtrack
        result=[]
        maxscore=max([V[n][ST]+Trans['E'][ST] for ST in V.index[-3:]])
        node=[ST for ST in V.index[-3:] if V[n][ST]+Trans['E'][ST]==maxscore][0]
        while True:
            prevnode=Backtrack[n][node][1]
            prevcol=Backtrack[n][node][0]
            result+=[node]
            node=prevnode
            n=prevcol
            if node in ['I0','S']:
                break
        return result[::-1]

import pandas as pd
import numpy as np

#file='input.txt'
#file='input2.txt'
#file='../../Downloads/dataset_11632_6.txt'
file='../../Downloads/rosalind_ba10g.txt'

f=open(file,'r')
lines=f.read().splitlines()
f.close()

x=list(lines[0])
th=float(map(str,lines[2].split(' '))[0])
pseudo=float(map(str,lines[2].split(' '))[-1])
alph=map(str,lines[4].split(' '))
aln=[list(line) for line in lines[6:]]

#print th,pseudo,alph,aln
print ' '.join(HMM_pseudo(th,pseudo,alph,aln).Align_to_HMMprofile(x))

    



M1 M2 M3 M4 M5 M6 M7 M8 M9 I9 M10 I10 I10 M11 M12 M13 M14 M15 I15 M16 M17 M18 M19 M20 I20 M21 M22 I22 M23 M24 I24 I24 M25 I25 M26 I26 I26 M27 I27 I27 I27 M28 M29 I29 I29 M30 M31 I31 M32
